In [17]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
from networkx.algorithms import bipartite
from tqdm import tqdm
#df = pd.read_excel('graph_測試.xlsx')

In [11]:
df = pd.read_excel('all_patent.xlsx')
invt_ID = df['invt_ID'].unique()

In [12]:
import math
n = math.ceil(len(invt_ID))

125267

In [13]:
df = df.dropna(how='all', axis=1)

In [18]:
#data is too big 拆成四個part跑(0,n),(n,2n),(2n,3n), (3n,len)
'''
inventor = []

for i in tqdm(range(len(invt_ID))):
    nodes_PN = []
    nodes_assignee = []
    edges = []
    for PN in df.loc[(df['invt_ID']==invt_ID[i])]['PN']:
        if PN not in nodes_PN:
            nodes_PN.append(PN)
    for ass_ID in df.loc[(df['invt_ID']== invt_ID[i])]['assignee_ID']:
        if ass_ID not in nodes_assignee:
            nodes_assignee.append(ass_ID)        
    for j in df.loc[(df['invt_ID']== invt_ID[i])].index:
        edges.append((df.loc[(df['invt_ID']==invt_ID[i])]['PN'][j],df.loc[(df['invt_ID']==invt_ID[i])]['assignee_ID'][j]))
    inventor.append({invt_ID[i]:{'nodes_PN':nodes_PN , 'nodes_assignee':nodes_assignee, 'edges':edges}})
'''

 14%|██████████▏                                                            | 18062/125267 [1:12:18<7:09:09,  4.16it/s]


KeyboardInterrupt: 

In [101]:
#nx.draw_networkx(B, with_labels=True)

In [7]:
from networkx.algorithms import bipartite
def get_connected_len(nodes_PN, nodes_assignee, edges):
    B = nx.Graph()
    B.add_nodes_from(nodes_PN,bipartite=0)
    B.add_nodes_from(nodes_assignee,bipartite=1)
    B.add_edges_from(edges)
    bipartite.is_bipartite(B)
    edges = B.edges()
    #nx.draw_networkx(B,pos = nx.drawing.layout.bipartite_layout(B, nodes_PN), width = 2)
    return len(list(nx.connected_components(B)))

In [8]:
disconnected = []
for i in range(len(invt_ID)):
    for key, graph in inventor[i].items():
        node_pn = graph['nodes_PN']
        node_ass = graph['nodes_assignee']
        edge = graph['edges']
        if get_connected_len(node_pn, node_ass, edge) > 1:
            #print(key)
            disconnected.append(key)

In [9]:
print(len(disconnected))
len(disconnected)

['fl:a_ln:abram-1',
 'fl:a_ln:acharya-2',
 'fl:a_ln:abouabdellah-1',
 'fl:a_ln:abraham-9',
 'fl:a_ln:ackerman-4',
 'fl:a_ln:adamis-1',
 'fl:a_ln:adejare-1']